In [2]:
import pandas as pd
import requests
import lxml.html as lh
r = requests.get('https://www.google.com/finance/quote/TSLA:NASDAQ?source=lnms&sa=X&ved=2ahUKEwjy9IK_-I_mAhW_QUEAHYXBBIUQ_AUoA3oECA4QCw&biw=1536&bih=722#scso=_zFThXbauNo_YwALfi5TgCA7:0', 
                 headers={"accept-language":"en"})

In [3]:
print(r.status_code)

200


In [4]:
# print the first 500 characters of the HTML
print(r.text[0:500])

<!doctype html><html lang="en" dir="ltr"><head><base href="https://www.google.com/finance/"><meta name="referrer" content="origin"><link rel="canonical" href="https://www.google.com/finance/quote/TSLA:NASDAQ"><meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no,minimal-ui"><meta name="google-site-verification" content="sxp7zFOFUzk09RdlFhuH2SoCn5nOkXgomiLeLIQ48p0"><meta name="mobile-web-app-capable" content="yes"><meta name="apple-mobi


In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
accion = soup.find_all('div', attrs={'class':'zzDege'})


In [6]:
results = soup.find_all('div', attrs={'class':'YMlKec fxKbKc'})
print('El precio actual de ' + accion[0].text + ' es: '+ results[0].text)

El precio actual de Tesla Inc es: $1,056.78


In [7]:
doc = lh.fromstring(r.content)
tr_elements = doc.xpath('//tr')

In [8]:
#Se comprueba que la tabla tiene 3 columnas y 9 filas (incluyendo los encabezados)
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3]

In [9]:
#Se crea una lista vacía
col=[]
i=0
#Para cada columns, se guarda el primer elemento (encabezado o header) y una lista vacía.
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 (USD)
2 Sep 2021infoFiscal Q3 2021 ended 9/30/21. Reported on 10/20/21.
3 Year/year change


In [10]:
#Dado que la primera fila está compuesta por los encabezados, los datos son guardados a partir de la segunda fila. 
for j in range(1,len(tr_elements)):
   
    #T es la j-ésima fila 
    T=tr_elements[j]
    
    #Si una fila no es de longitud 3, el elemento //tr no es de esta tabla.  
    if len(T)!=3:
        break
    
    #i es el indice de la columna
    i=0
    
    #Se itera sobre cada elemento de la fila.
    for t in T.iterchildren():
        data=t.text_content() 
        #Comprobación de si la fila está vacía.
        if i>0:
        #Se convierte cualquier valor numérico a entero.
            try:
                data=int(data)
            except:
                pass
        #Se añaden los datos a la lista vacía.
        col[i][1].append(data)
        #Se incrementa en i para la próxima columna. 
        i+=1

In [11]:
#Sin tomar en cuanta los encabezados, la tabla tiene 3 columnas y 8 filas
[len(C) for (title,C) in col]

[8, 8, 8]

In [12]:
#Se definen los encabezados de las columnas y se crea el DataFrame
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [13]:
tabla= df.set_index('(USD)')

In [14]:
header_fecha=col[1][0][0:8]

In [15]:
#Se renombra la columna del medio
header_fecha=col[1][0][0:8]
tabla.rename({col[1][0]: header_fecha}, axis=1, inplace=True)
tabla

,Sep 2021,Year/year change
(USD),,
Revenue,13.76B,56.85%
Net income,1.62B,388.82%
Diluted EPS,1.44,433.33%
Net profit margin,11.76%,211.94%
Operating income,2.06B,168.28%
Net change in cash,-131.00M,-102.22%
Cash on hand,16.06B,10.56%
Cost of revenue,10.10B,50.52%


In [16]:
#Empresas comparables
comparables = soup.find('div', attrs={'class':'bF2Mne'})

In [17]:
#Se muestran por separado y en una lista todas las empresas comparables
empresas = comparables.select(".qIEjSe")
empresas = [x.get_text() for x in empresas]
empresas

['Amazon.com, Inc.',
 'Apple Inc',
 'NVIDIA Corporation',
 'Lucid Group Inc',
 'Ford Motor Company',
 'Palantir Technologies Inc',
 'Microsoft Corporation',
 'Rivian Automotive Inc',
 'SPDR S&P 500 ETF Trust',
 'Nio Inc - ADR']

In [18]:
#Se hace lo mismo pero con los precios y las variaciones diarias
precio_comp = comparables.select(".Z63m9d")
precio_comp = [x.get_text() for x in precio_comp]
precio_comp

['$3,334.34',
 '1.14%',
 '$177.57',
 '0.35%',
 '$294.11',
 '0.59%',
 '$38.05',
 '1.81%',
 '$20.77',
 '1.47%',
 '$18.21',
 '2.57%',
 '$336.32',
 '0.88%',
 '$103.69',
 '0.26%',
 '$474.96',
 '0.25%',
 '$31.68',
 '2.28%']

In [19]:
#Los precios y las variaciones vienen intercalados en la misma lista, se procede a separarlos en listas distintas
porcentaje = []
for precio in precio_comp:
    if "%" in precio:
        porcentaje.append(precio)
        precio_comp.remove(precio)

In [20]:
print(porcentaje)
print(precio_comp)

['1.14%', '0.35%', '0.59%', '1.81%', '1.47%', '2.57%', '0.88%', '0.26%', '0.25%', '2.28%']
['$3,334.34', '$177.57', '$294.11', '$38.05', '$20.77', '$18.21', '$336.32', '$103.69', '$474.96', '$31.68']


In [21]:
ticker = comparables.select(".lUuFj")
ticker = [x.get_text() for x in ticker]
ticker 

['AMZN', 'AAPL', 'NVDA', 'LCID', 'F', 'PLTR', 'MSFT', 'RIVN', 'SPY', 'NIO']

In [22]:
#Se crea un DataFrame con la información de las empresas comparables
empresas_comp = pd.DataFrame({'Comparables':empresas, 'Ticker':ticker, 'Precio Actual':precio_comp,'Variación':porcentaje})
empresas_comp

,Comparables,Ticker,Precio Actual,Variación
0,"Amazon.com, Inc.",AMZN,"$3,334.34",1.14%
1,Apple Inc,AAPL,$177.57,0.35%
2,NVIDIA Corporation,NVDA,$294.11,0.59%
3,Lucid Group Inc,LCID,$38.05,1.81%
4,Ford Motor Company,F,$20.77,1.47%
5,Palantir Technologies Inc,PLTR,$18.21,2.57%
6,Microsoft Corporation,MSFT,$336.32,0.88%
7,Rivian Automotive Inc,RIVN,$103.69,0.26%
8,SPDR S&P 500 ETF Trust,SPY,$474.96,0.25%
9,Nio Inc - ADR,NIO,$31.68,2.28%


In [23]:
#Noticias de la acción
noticias = soup.find('div', attrs={'class':'Gfxi4'})

In [24]:
periodico = noticias.select(".sfyJob")
periodico = [x.get_text() for x in periodico]
periodico


['Business Standard',
 'CNBC',
 "Barron's",
 "Investor's Business Daily",
 'Fox Business',
 'Bloomberg.com']

In [25]:
month = noticias.select(".Adak")
month = [x.get_text() for x in month]
month


['12 hours ago',
 '23 hours ago',
 '1 day ago',
 '1 hour ago',
 '23 hours ago',
 '1 day ago']

In [26]:
titular = noticias.select(".Yfwt5")
titular = [x.get_text() for x in titular]
titular


['Tesla CEO Elon Musk advises young people to learn as much as possible',
 'Tesla asks employees to fight policy proposal that would make rooftop solar \nmore expensive in California',
 "Tesla Could 'Considerably' Extend EV Lead. Deutsche Bank Lifts Target to \n$1,200.",
 'Dow Jones Futures: 2022 Market Rally? Bring It On; Xpeng, Nio EV Deliveries \nStrong With Tesla On Tap',
 'How Elon Musk’s software focus helped Tesla navigate chip shortage',
 "Tesla Deliveries: Elon Musk's TSLA Races to Meet New Years Eve Deadline"]

In [27]:
links = soup.find_all("div", {'class':'yY3Lee'})

In [28]:
#El anchor <a> se usa para crear enlaces a otras páginas web. La propiedad 'href' define su ubicación. 
link = [x.a.get("href") for x in links]
link

['https://www.business-standard.com/article/international/tesla-ceo-elon-musk-advices-young-people-to-learn-as-much-as-possible-122010100291_1.html',
 'https://www.cnbc.com/2021/12/31/tesla-asks-employees-to-fight-california-plan-to-cut-solar-incentives.html',
 'https://www.barrons.com/articles/tesla-tsla-stock-electric-vehicles-deliveries-price-target-51640958938',
 'https://www.investors.com/market-trend/stock-market-today/dow-jones-futures-tesla-nio-ev-deliveries-in-focus-2022-market-rally/',
 'https://www.foxbusiness.com/technology/elon-musks-software-focus-helped-tesla-navigate-chip-shortage',
 'https://www.bloomberg.com/news/articles/2021-12-31/tesla-has-busy-new-year-s-eve-plans-in-push-to-record-deliveries']

In [29]:
noticias = pd.DataFrame({'Periódico':periodico, 'Fecha':month, 'Titular':titular,'URL':link})
noticias

,Periódico,Fecha,Titular,URL
0,Business Standard,12 hours ago,Tesla CEO Elon Musk advises young people to le...,https://www.business-standard.com/article/inte...
1,CNBC,23 hours ago,Tesla asks employees to fight policy proposal ...,https://www.cnbc.com/2021/12/31/tesla-asks-emp...
2,Barron's,1 day ago,Tesla Could 'Considerably' Extend EV Lead. Deu...,https://www.barrons.com/articles/tesla-tsla-st...
3,Investor's Business Daily,1 hour ago,Dow Jones Futures: 2022 Market Rally? Bring It...,https://www.investors.com/market-trend/stock-m...
4,Fox Business,23 hours ago,How Elon Musk’s software focus helped Tesla na...,https://www.foxbusiness.com/technology/elon-mu...
5,Bloomberg.com,1 day ago,Tesla Deliveries: Elon Musk's TSLA Races to Me...,https://www.bloomberg.com/news/articles/2021-1...


In [30]:
#Tabla resumen con datos básicos de la acción
tabla_resumen = soup.find('div', attrs={'class':'eYanAe'})

In [31]:
valores = tabla_resumen.select(".P6K39c")
valores = [x.get_text() for x in valores]
valores

['$1,070.34',
 '$1,054.59 - $1,082.00',
 '$539.49 - $1,243.49',
 '1.06T USD',
 '24.03M',
 '342.92',
 '-',
 'NASDAQ']

In [32]:
campos = tabla_resumen.select(".mfs7Fc")
campos = [x.get_text() for x in campos]
campos


['Previous close',
 'Day range',
 'Year range',
 'Market cap',
 'Volume',
 'P/E ratio',
 'Dividend yield',
 'Primary exchange']

In [33]:
#Se usa "options.display.html.use_mathjax=False" debido a que sin esto, los números que se encuentran entre 
#signos de dolar ($) vienen en un  formato diferente.
pd.options.display.html.use_mathjax=False
resumen_financials = pd.DataFrame({'Descripción':campos, accion[0].text:valores})
resumen_financials= resumen_financials.set_index('Descripción')
resumen_financials 

,Tesla Inc
Descripción,
Previous close,"$1,070.34"
Day range,"$1,054.59 - $1,082.00"
Year range,"$539.49 - $1,243.49"
Market cap,1.06T USD
Volume,24.03M
P/E ratio,342.92
Dividend yield,-
Primary exchange,NASDAQ


### Datrames finales

In [34]:
tabla

,Sep 2021,Year/year change
(USD),,
Revenue,13.76B,56.85%
Net income,1.62B,388.82%
Diluted EPS,1.44,433.33%
Net profit margin,11.76%,211.94%
Operating income,2.06B,168.28%
Net change in cash,-131.00M,-102.22%
Cash on hand,16.06B,10.56%
Cost of revenue,10.10B,50.52%


In [35]:
empresas_comp

,Comparables,Ticker,Precio Actual,Variación
0,"Amazon.com, Inc.",AMZN,"$3,334.34",1.14%
1,Apple Inc,AAPL,$177.57,0.35%
2,NVIDIA Corporation,NVDA,$294.11,0.59%
3,Lucid Group Inc,LCID,$38.05,1.81%
4,Ford Motor Company,F,$20.77,1.47%
5,Palantir Technologies Inc,PLTR,$18.21,2.57%
6,Microsoft Corporation,MSFT,$336.32,0.88%
7,Rivian Automotive Inc,RIVN,$103.69,0.26%
8,SPDR S&P 500 ETF Trust,SPY,$474.96,0.25%
9,Nio Inc - ADR,NIO,$31.68,2.28%


In [36]:
noticias

,Periódico,Fecha,Titular,URL
0,Business Standard,12 hours ago,Tesla CEO Elon Musk advises young people to le...,https://www.business-standard.com/article/inte...
1,CNBC,23 hours ago,Tesla asks employees to fight policy proposal ...,https://www.cnbc.com/2021/12/31/tesla-asks-emp...
2,Barron's,1 day ago,Tesla Could 'Considerably' Extend EV Lead. Deu...,https://www.barrons.com/articles/tesla-tsla-st...
3,Investor's Business Daily,1 hour ago,Dow Jones Futures: 2022 Market Rally? Bring It...,https://www.investors.com/market-trend/stock-m...
4,Fox Business,23 hours ago,How Elon Musk’s software focus helped Tesla na...,https://www.foxbusiness.com/technology/elon-mu...
5,Bloomberg.com,1 day ago,Tesla Deliveries: Elon Musk's TSLA Races to Me...,https://www.bloomberg.com/news/articles/2021-1...


In [37]:
resumen_financials

,Tesla Inc
Descripción,
Previous close,"$1,070.34"
Day range,"$1,054.59 - $1,082.00"
Year range,"$539.49 - $1,243.49"
Market cap,1.06T USD
Volume,24.03M
P/E ratio,342.92
Dividend yield,-
Primary exchange,NASDAQ
